In [2]:
import pandas as pd
import geopandas as gpd
import json

import bokeh
from bokeh import plotting
from bokeh.models import FactorRange
from bokeh.palettes import Category20c
from bokeh.layouts import column, row, widgetbox
from bokeh.io import save, show, output_file, output_notebook, reset_output, export_png
from bokeh.plotting import figure
from bokeh.io.doc import curdoc
from bokeh.models import (
    GeoJSONDataSource, ColumnDataSource, ColorBar, Slider, Spacer,
    HoverTool, TapTool, Panel, Tabs, Legend, Toggle, LegendItem, Button, Select, CategoricalColorMapper
)
from bokeh.palettes import brewer
from bokeh.models.widgets import Div
from matplotlib import pyplot as plt
from matplotlib.colors import rgb2hex

# Choropleth map
In this section we present the code used to generate the choropleth map. We have found inspiration and help on the following sites:
- https://jimking100.github.io/2019-09-04-Post-3/ 
- https://cbouy.github.io/2019/06/09/interactive-map.html

Some of the code has been taken directly from Cédric Bouysset (cbouy), and then only been modified slightly. We will indicate these pieces of code with an inline comment.

Please note that all interactive elements only work if the code is run on a bokeh server. We have hosted the plot as an app on Heroku, and deployed it with github in order to display it on our website. The actual app can be found on https://firedepartment-sanfran.herokuapp.com/Map3.

In [3]:
# Import all data sets we made earlier
nhood_all = pd.read_csv('nhood_all.csv')
nhood_priority = pd.read_csv('nhood_priority.csv')
nhood_medical = pd.read_csv('nhood_medical.csv')

district_all = pd.read_csv('district_all.csv')
district_priority = pd.read_csv('district_priority.csv')
district_medical = pd.read_csv('district_medical.csv')

In [4]:
district_all = district_all.astype({'District': 'object'})
district_priority = district_priority.astype({'District': 'object'})
district_medical = district_medical.astype({'District': 'object'})

In [5]:
# Load geojson file for neighborhoods
nhood = gpd.read_file("Neighborhoods.geojson")
nhood = nhood.rename(columns={'nhood':'Neighborhood'}).set_geometry('geometry')
nhood.crs = {'init': 'epsg:4326'}

In [6]:
# Load geojson file for districts, and clean it up
district = gpd.read_file("Districts.geojson")
district = district.rename(columns={'supervisor':'District'}).set_geometry('geometry')
district = district.drop(columns=['supname', 'supdist', 'numbertext', 'supdistpad'])
district.sort_values(by=["District"])
district = district.astype({'District': 'int32'})
district.sort_values(by=["District"], inplace=True)
district.crs = {'init': 'epsg:4326'}

In [7]:
# Split dataframe containing high and low priority calls for districts, such that we can assign different bins
district_all.sort_values(by=["District"], inplace=True)
district_priority2 = district_priority[district_priority["Priority"] == 2]
district_priority3 = district_priority[district_priority["Priority"] == 3]

In the following, we define the bins and their labels. We define two different sets of bins, as some of the values are much larger than others. The bins are assigned to the data in the following way:
- nhood_all: bins1
- nood_priority: bins1
- nhood_medical: bins1
- district_all: bins
- district_priority2: bins1
- district_priority3: bins
- district_medical: bins1

In [8]:
# Create bins
bins = [0,10000,20000,30000,40000,60000,90000,100000]
# Create labels - from cbouy
bin_labels = [f'≤{bins[1]}'] + [f'{bins[i]}-{bins[i+1]}' for i in range(1,len(bins)-2)] + [f'>{bins[-2]}']

In [9]:
# Create bins
bins1 = [0,2000,5000,10000,15000,20000,30000,50000,76000]
# Create labels - from cbouy
bin_labels1 = [f'≤{bins1[1]}'] + [f'{bins1[i]}-{bins1[i+1]}' for i in range(1,len(bins1)-2)] + [f'>{bins1[-2]}']

In [10]:
# Assign each entry to a bin - from cbouy
nhood_all['bin'] = pd.cut(
    nhood_all['Calls'], bins=bins1, right=True, include_lowest=True, precision=0, labels=bin_labels1,
).astype(str)

nhood_priority['bin'] = pd.cut(
    nhood_priority['Calls'], bins=bins1, right=True, include_lowest=True, precision=0, labels=bin_labels1,
).astype(str)

nhood_medical['bin'] = pd.cut(
    nhood_medical['Calls'], bins=bins1, right=True, include_lowest=True, precision=0, labels=bin_labels1,
).astype(str)

district_all['bin'] = pd.cut(
    district_all['Calls'], bins=bins, right=True, include_lowest=True, precision=0, labels=bin_labels,
).astype(str)

district_priority2['bin'] = pd.cut(
    district_priority2['Calls'], bins=bins1, right=True, include_lowest=True, precision=0, labels=bin_labels1,
).astype(str)

district_priority3['bin'] = pd.cut(
    district_priority3['Calls'], bins=bins, right=True, include_lowest=True, precision=0, labels=bin_labels,
).astype(str)

district_medical['bin'] = pd.cut(
    district_medical['Calls'], bins=bins1, right=True, include_lowest=True, precision=0, labels=bin_labels1,
).astype(str)

/Users/Nanorii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Nanorii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
# Merge the geographic neighborhood data with fire department data
nhood_a = pd.merge(nhood, nhood_all, on='Neighborhood', how='left')
nhood_p = pd.merge(nhood, nhood_priority, on='Neighborhood', how='left')
nhood_m = pd.merge(nhood, nhood_medical, on='Neighborhood', how='left')

# Merge the geographic district data with fire department data
district_a = pd.merge(district, district_all, on='District', how='left')
district_p2 = pd.merge(district, district_priority2, on='District', how='left')
district_p3 = pd.merge(district, district_priority3, on='District', how='left')
district_m = pd.merge(district, district_medical, on='District', how='left')

Next, we define two color palettes and assign each entry a color, based on the bin this entry was assigned previously. 

In [12]:
# Define color palettes
palette = brewer['YlOrRd'][len(bins)-1]
palette1 = brewer['YlOrRd'][len(bins1)-1]

# Reverse color palettes
palette = palette[::-1]
palette1 = palette1[::-1]

# From cbouy
def val_to_color(value, nan_color='#d9d9d9'):
    if isinstance(value, str): return nan_color
    for i in range(1,len(bins)):
        if value <= bins[i]:
            return palette[i-1]
        
def val_to_color1(value, nan_color='#d9d9d9'):
    if isinstance(value, str): return nan_color
    for i in range(1,len(bins1)):
        if value <= bins1[i]:
            return palette1[i-1]

# Assign number of calls to a color
nhood_a['color'] = nhood_a['Calls'].apply(val_to_color1)
nhood_p['color'] = nhood_p['Calls'].apply(val_to_color1)
nhood_m['color'] = nhood_m['Calls'].apply(val_to_color1)

district_a['color'] = district_a['Calls'].apply(val_to_color)
district_m['color'] = district_m['Calls'].apply(val_to_color)
district_p2['color'] = district_p2['Calls'].apply(val_to_color1)
district_p3['color'] = district_p3['Calls'].apply(val_to_color)

We define y-coordinates for the colorbar, such that we can display it as a nice vertical bar. Furthermore, the width of the color bar is set to 4.7.

In [13]:
# From cbouy
def bin_to_cbar_y(value):
    for i,b in enumerate(bin_labels):
        if value == b:
            return 5*(i+1)
        
def bin_to_cbar_y1(value):
    for i,b in enumerate(bin_labels1):
        if value == b:
            return 5*(i+1)
        
# Assign y coordinates  
nhood_a['cbar_y'] = nhood_a['bin'].apply(bin_to_cbar_y1)
nhood_p['cbar_y'] = nhood_p['bin'].apply(bin_to_cbar_y1)
nhood_m['cbar_y'] = nhood_m['bin'].apply(bin_to_cbar_y1)

district_a['cbar_y'] = district_a['bin'].apply(bin_to_cbar_y)
district_m['cbar_y'] = district_m['bin'].apply(bin_to_cbar_y1)
district_p2['cbar_y'] = district_p2['bin'].apply(bin_to_cbar_y1)
district_p3['cbar_y'] = district_p3['bin'].apply(bin_to_cbar_y)

# Assign width
nhood_a['cbar_w'] = 4.7
nhood_p['cbar_w'] = 4.7
nhood_m['cbar_w'] = 4.7

district_a['cbar_w'] = 4.7
district_m['cbar_w'] = 4.7
district_p2['cbar_w'] = 4.7
district_p3['cbar_w'] = 4.7

Next, we define the interactive tools. We want a slider that enables us to slide through the years, as well as a drop-down menu that enables us to choose different criteria.

In [14]:
# Sliders
slider = Slider(start=2000, end=2019, value=2010, step=1, title="Year")
slider1 = Slider(start=2000, end=2019, value=2010, step=1, title="Year")

# Drop-down menus
select = Select(title='Select criteria', value='All calls', options=['All calls', 'High priority calls',
                                                                    'Low priority calls', 'Medical incident calls'])
select1 = Select(title='Select criteria', value='All calls', options=['All calls', 'High priority calls',
                                                                    'Low priority calls', 'Medical incident calls'])

The following two functions will be used to generate the source data for the maps, whenever the data needs to be updated.

In [15]:
# Function that returns neighborhood json_data for the selected year
def json_data(selectedYear):
    yr = selectedYear
    cr = select.value
    
    # Pull correct data
    if cr == "High priority calls":
        df = nhood_p[(nhood_p['year'] == yr) & (nhood_p['Priority'] == 3)]
    elif cr == "Low priority calls":
        df = nhood_p[(nhood_p['year'] == yr) & (nhood_p['Priority'] == 2)]
    elif cr == "Medical incident calls":
        df = nhood_m[nhood_m['year'] == yr]
    elif cr == "All calls":
        df = nhood_a[nhood_a['year'] == yr]

    # Convert to json
    df_json = json.loads(df.to_json())

    # Convert to json preferred string-like object 
    json_data = json.dumps(df_json)
    return json_data

In [16]:
# Function that returns district json_data for the selected year 
def json_data1(selectedYear):
    yr = selectedYear
    cr = select1.value
    
    # Pull correct data
    if cr == "High priority calls":
        df = district_p3[district_p3['year'] == yr]
    elif cr == "Low priority calls":
        df = district_p2[district_p2['year'] == yr]
    elif cr == "Medical incident calls":
        df = district_m[district_m['year'] == yr]
    elif cr == "All calls":
        df = district_a[district_a['year'] == yr]

    # Convert to json
    df_json = json.loads(df.to_json())

    # Convert to json preferred string-like object 
    json_data1 = json.dumps(df_json)
    return json_data1

Now we can define the "source data" for the two maps. This data will be updated whenever the value of the slider or drop-down menu changes.

In [17]:
# Source that will contain all necessary data for the map
nhood_src = GeoJSONDataSource(geojson=json_data(2010))
district_src = GeoJSONDataSource(geojson=json_data1(2010))

Next we make the functions that generate the actual maps. We make a hover tool that displays the name of the neighborhood/district as well as the number of calls, whenever we hover over a certain neighborhood/district. We also ensure that a neighborhood/district is highlighted with an outline when we hover over it.

In [30]:
# Create map with neighborhoods
def make_nmap():
    cr = select.value
    
    p = figure(
        title = cr + ' to fire department by neighborhood in San Francisco in 2010',
        plot_height=700 , plot_width=650,
        toolbar_location=None, tools="tap,pan,wheel_zoom,box_zoom,save,reset", toolbar_sticky=False,
        active_scroll="wheel_zoom",
    )
    p.title.text_font_size = '12pt'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    
    map_hover = HoverTool(tooltips=[
    ('Neighborhood','@Neighborhood'),
    ('Number of calls', '@Calls')
    ])

    # Add hover tool
    p.add_tools(map_hover)

    # Add patches (neighborhoods) to the figure - from cbouy
    patches = p.patches(
        'xs','ys', source=nhood_src,
        fill_color='color',
        line_color='black', line_width=0.25, fill_alpha=1,
        hover_fill_color='color',
    )
    # Outline when we hover over a neighborhood - from cbouy
    patches.hover_glyph.line_color = '#3bdd9d'
    patches.hover_glyph.line_width = 3
    patches.nonselection_glyph = None
    
    return p

In [31]:
# Create map with districts 
def make_dmap():
    cr = select1.value
    
    f = figure(
        title = cr + ' to fire department by district in San Francisco in 2010',
        plot_height=700 , plot_width=650,
        toolbar_location=None, tools="tap,pan,wheel_zoom,box_zoom,save,reset", toolbar_sticky=False,
        active_scroll="wheel_zoom",
    )
    f.title.text_font_size = '12pt'
    f.xgrid.grid_line_color = None
    f.ygrid.grid_line_color = None
    f.axis.visible = False
    
    map_hover = HoverTool(tooltips=[
    ('District','@District'),
    ('Number of calls', '@Calls')
    ])

    # Add hover tool
    f.add_tools(map_hover)

    # Add patches (districts) to the figure - from cbouy
    patches = f.patches(
        'xs','ys', source=district_src,
        fill_color='color',
        line_color='black', line_width=0.25, fill_alpha=1,
        hover_fill_color='color',
    )
    # outline when we hover over a district - from cbouy
    patches.hover_glyph.line_color = '#3bdd9d'
    patches.hover_glyph.line_width = 3
    patches.nonselection_glyph = None
    
    return f

Now we need to make the colorbars for the two maps. The rectangles are made with the cbar_y, cbar_w and color columns in the dataframes. We make sure that all patches of a given color are highligted whenever we hover over that specific color on the colorbar.

In [32]:
# Create interactive colorbar for the neighborhood map
def make_nbar():
    p_bar = figure(
        title=None, plot_height=400 , plot_width=130,
        tools="tap", toolbar_location=None, match_aspect=True,
    )
    p_bar.xgrid.grid_line_color = None
    p_bar.ygrid.grid_line_color = None
    p_bar.outline_line_color = None
    p_bar.xaxis.visible = False 
    
    cr = select.value
    if cr == "All calls":
        df = nhood_a
    elif cr == "High priority calls" or cr == "Low priority calls":
        df = nhood_p
    elif cr == "Medical incident calls":
        df = nhood_m

    # Set the title and ticks of the colorbar
    p_bar.yaxis.axis_label = "Number of calls"
    p_bar.yaxis.ticker = sorted(df['cbar_y'].unique())
    p_bar.yaxis.major_label_overrides = dict([(i[0],i[1]) for i in df.groupby(['cbar_y','bin']).describe().index])
    p_bar.yaxis.axis_label_text_font_size = "10pt"
    p_bar.yaxis.major_label_text_font_size = "8pt"

    # Activate hover
    hover_bar = HoverTool(tooltips=None)
    p_bar.add_tools(hover_bar)

    # Plot the rectangles for the colorbar
    cbar = p_bar.rect(x=0, y='cbar_y', width=1.7, height='cbar_w',
        color='color', source=nhood_src,
        hover_line_color='#3bdd9d', hover_fill_color='color')

    # Outline when hovering over the colorbar legend
    cbar.hover_glyph.line_width = 4
    cbar.nonselection_glyph = None
    
    return p_bar

In [33]:
# Create interactive colorbar for the district map
def make_dbar():
    f_bar = figure(
        title=None, plot_height=400 , plot_width=130,
        tools="tap", toolbar_location=None, match_aspect=True,
    )
    f_bar.xgrid.grid_line_color = None
    f_bar.ygrid.grid_line_color = None
    f_bar.outline_line_color = None
    f_bar.xaxis.visible = False 
    
    cr = select1.value
    if cr == "All calls":
        df = district_a
    elif cr == "High priority calls":
        df = district_p3
    elif cr == "Low priority calls":
        df = district_p2
    elif cr == "Medical incident calls":
        df = district_m

    # Set the title and ticks of the colorbar
    f_bar.yaxis.axis_label = "Number of calls"
    f_bar.yaxis.ticker = sorted(df['cbar_y'].unique())
    f_bar.yaxis.major_label_overrides = dict([(i[0],i[1]) for i in df.groupby(['cbar_y','bin']).describe().index])
    f_bar.yaxis.axis_label_text_font_size = "10pt"
    f_bar.yaxis.major_label_text_font_size = "8pt"

    # Activate hover
    hover_bar = HoverTool(tooltips=None)
    f_bar.add_tools(hover_bar)

    # Plot the rectangles for the colorbar
    cbar = f_bar.rect(x=0, y='cbar_y', width=1.7, height='cbar_w',
        color='color', source=district_src,
        hover_line_color='#3bdd9d', hover_fill_color='color')

    # Outline when hovering over the colorbar legend
    cbar.hover_glyph.line_width = 4
    cbar.nonselection_glyph = None
    
    return f_bar

In the following we define the callback functions that define the behaviour when using the sliders and drop-down menus. When a slider is moved, we simply want the source data to update. However, when a new criteria is selected from the drop-down, we make the entire map and colorbar again, with the new source data. 

In [34]:
def callback_sli(attr, old, new):
    yr = slider.value
    cr = select.value

    new_data = json_data(yr)
    
    # Update the data
    p.title.text = cr + " to fire department by neighborhood in San Francisco in " + str(yr)
    nhood_src.geojson = new_data

In [35]:
def callback_sli1(attr, old, new):
    yr = slider1.value
    cr = select1.value

    new_data = json_data1(yr)
    
    # Update the data
    f.title.text = cr + " to fire department by district in San Francisco in " + str(yr)
    district_src.geojson = new_data

In [36]:
# Function to update the neighborhood map when selected criteria is changed
def callback_sel(attr, old, new):
    yr = slider.value
    cr = select.value
    
    # Update the data
    new_data = json_data(yr)
    nhood_src.geojson = new_data

    # Make map and colorbar again
    p = make_nmap()
    p_bar = make_nbar()
    
    p.title.text = cr + " to fire department by neighborhood in San Francisco"

    tab_n = Panel(title="Neighborhoods",
    child=row(
        p_bar, 
        column(p, 
            row(widgetbox(slider),
            widgetbox(select)))
    ))

    tabs = Tabs(tabs=[ tab_n, tab_d ])
    
    footer = Div(text="""
    Data: <a href="https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3"> 
    Fire Department Calls for Service</a>""")
    
    # Clear curdoc and add new layout to curdoc
    layout = column(tabs, footer)
    curdoc().clear()
    curdoc().add_root(layout)

In [37]:
# Function to update the district map when selected criteria is changed
def callback_sel1(attr, old, new):
    yr = slider1.value
    cr = select1.value
    
    # Update the data
    new_data = json_data1(yr)
    district_src.geojson = new_data
    
    # Make map and colorbar again
    f = make_dmap()
    f_bar = make_dbar()
    
    f.title.text = cr + " to fire department by district in San Francisco"

    tab_d = Panel(title="Districts",
    child=row(
        f_bar, 
        column(f, 
            row(widgetbox(slider1),
            widgetbox(select1)))
    ))

    tabs = Tabs(tabs=[ tab_n, tab_d ])
    tabs.active=1
    
    footer = Div(text="""
    Data: <a href="https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3"> 
    Fire Department Calls for Service</a>""")
    
    # Clear curdoc and add new layout to curdoc
    layout = column(tabs, footer)
    curdoc().clear()
    curdoc().add_root(layout)

Finally, we draw the maps and colorbars, and build the desired layout. The layout is then added to the current document, and can be displayed on a bokeh server.

In [40]:
# Use callback functions when sliders are used 
slider.on_change('value', callback_sli)
slider1.on_change('value', callback_sli1)

# Use callback functions when drop-downs are used 
select.on_change('value', callback_sel)
select1.on_change('value', callback_sel1)

# Make neighborhood map and colorbar
p = make_nmap()
p_bar = make_nbar()

# Make tab containing neighborhood map + tools
tab_n = Panel(title="Neighborhoods",
    child=row(
        p_bar, 
        column(p,
            row(widgetbox(slider),
            widgetbox(select)))
    ))

# Make district map and colorbar
f = make_dmap()
f_bar = make_dbar()

# Make tab containing district map + tools
tab_d = Panel(title="Districts",
    child=row(
        f_bar, 
        column(f, 
            row(widgetbox(slider1),
            widgetbox(select1)))
    ))

# Define tabs 
tabs = Tabs(tabs=[ tab_n, tab_d])


In [41]:
# Add layout to current document
footer = Div(text="""
Data: <a href="https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3"> Fire Department Calls for Service</a>
""")
layout = column(tabs, footer)
curdoc().add_root(layout)